In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
state = ['ELASTIC', 'EFFECTIVE', 'MOMENTUM', 'IONIZATION', 'ATTACHMENT', 'EXCITATION']

In [ ]:
filePath = '/content/drive/MyDrive/KAUST/bolsig/bolsigdbN2O2.txt'

In [ ]:
columns = ['state', 'equation', 'value', 'species', 'process', 'param.', 'comment', 'updated']

In [ ]:
f = open(filePath, 'r')
data = {c : list() for c in columns}

lines = f.readlines()
lineIdx = 0

while lineIdx < len(lines):
  line = lines[lineIdx]
  line = line.strip()

  if line in state:
    exist = {c : 0 for c in columns}

    for i in range(3):
      if ':' in line: break
      data[columns[i]].append(line)
      exist[columns[i]] = 1

      lineIdx += 1
      line = lines[lineIdx].strip()

    commentNum = 0
    while True:
      if 'COLUMNS' in line:
        break

      idx = line.find(':')
      key = line[:idx].strip().lower()
      val = line[idx+1:].strip()

      if len(data['state']) == len(data[key]):
        data[key][-1] = data[key][-1] + ' ' + val
      else:
        data[key].append(val)
        exist[key] = 1

      lineIdx += 1
      line = lines[lineIdx].strip()

    for c in columns:
      if exist[c] == 0:
        data[c].append('')

  lineIdx += 1
f.close()

df = pd.DataFrame(data)
df['id'] = df.index + 1
df.insert(0, 'id', df.pop('id'))
df.to_csv(filePath.split('.')[0] + '(group).csv', index = False)

In [ ]:
def findOption(eq):
  option = None

  if '<->' in eq:
    idx = eq.find('<')
    option = [eq[:idx].strip(), eq[idx+3:].strip()]
  elif '->' in eq:
    idx = eq.find('>')
    option = [eq[:idx-1].strip()]
  else:
    option = [eq.strip()]

  return option

'''
species : not include ()
detailed : include ()

N2 -> N2* : N2
N2 -> N2(*) : N2
N2* -> N2(*) : N2*
N2(*) -> N2 : N2(*)
N2(*) -> N2(**) : N2(*)

N2 <-> N2* : N2, N2*
N2* <-> N2(*): N2*, N2(*)
N2(*) <-> N2* : N2(*), N2*
N2(*) <-> N2*(**) : N2(*), N2*(**)

more than 2 => All species
'''

optionDf = df.copy()
countData = dict()

for idx, row in optionDf.iterrows():
  options = findOption(row['equation'])

  for i, option in enumerate(options):
    if option in countData:
      countData[option] += 1
    else:
      countData[option] = 1

    optionDf.loc[idx, f'option{i+1}'] = option

_countData = {'option' : list(countData.keys()), 'count' : list(countData.values())}
countDf = pd.DataFrame(_countData)
countDf['id'] = countDf.index + 1
countDf.insert(0, 'id', countDf.pop('id'))

countDf.to_csv(filePath.split('.')[0] + '(count).csv', index = False)
optionDf.to_csv(filePath.split('.')[0] + '(option).csv', index = False)

In [ ]:
countDf.T

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
id,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
option,N2,N2(V1),N2(V2),N2(V3),N2(V4),N2(V5),N2(V6),N2(V7),N2(V8),N2(V9),...,O,O(1D),O(1S),O(4S0),O(2D0),O(2P0),O(3P0),O3,O2(a1),O2(b1)
count,22,1,1,1,1,1,1,1,1,1,...,8,1,1,1,1,1,1,4,6,5


In [ ]:
choosedOptions = [14, 1]
choosedOptions.sort()
choosedDf = pd.DataFrame()
tempDf = None

for idx in choosedOptions:
  option = countDf[countDf.id == idx].iloc[0]['option']

  tempDf = optionDf[(optionDf.option1 == option) | (optionDf.option2 == option)].copy()
  tempDf['criteria'] = option
  tempDf.insert(0, 'criteria', tempDf.pop('criteria'))

  choosedDf = pd.concat([choosedDf, tempDf])

choosedDf.reset_index(drop=True, inplace=True)
choosedDf['no'] = choosedDf.index + 1
choosedDf.insert(0, 'no', choosedDf.pop('no'))
choosedDf.to_csv(filePath.split('.')[0] + '(choosed).csv', index = False)

In [ ]:
choosedDf

,no,criteria,id,state,equation,value,species,process,param.,comment,updated,option1,option2
0,1,N2,1,ELASTIC,N2 -> N2,1.950000e-5,e / N2,"E + N2 -> E + N2, Elastic","m/M = 0.0000195, complete set",elastic MOMENTUM-TRANSFER CROSS SECTION.,2012-10-11 09:34:55,N2,NaN
1,2,N2,2,EXCITATION,N2 <-> N2(V1),2.889000e-1,e / N2,"E + N2 -> E + N2( VIB V1), Excitation","E = 0.2889 eV, complete set",,2012-10-11 09:34:55,N2,N2(V1)
2,3,N2,3,EXCITATION,N2 <-> N2(V2),5.742000e-1,e / N2,"E + N2 -> E + N2( VIB 2V1), Excitation","E = 0.5742 eV, complete set",,2012-10-11 09:34:55,N2,N2(V2)
3,4,N2,4,EXCITATION,N2 <-> N2(V3),8.559000e-1,e / N2,"E + N2 -> E + N2( VIB 3V1), Excitation","E = 0.8559 eV, complete set",,2012-10-11 09:34:55,N2,N2(V3)
4,5,N2,5,EXCITATION,N2 <-> N2(V4),1.134200e+0,e / N2,"E + N2 -> E + N2( VIB 4V1), Excitation","E = 1.1342 eV, complete set",,2012-10-11 09:34:55,N2,N2(V4)
5,6,N2,6,EXCITATION,N2 <-> N2(V5),1.408800e+0,e / N2,"E + N2 -> E + N2( VIB 5V1), Excitation","E = 1.4088 eV, complete set",,2012-10-11 09:34:55,N2,N2(V5)
6,7,N2,7,EXCITATION,N2 <-> N2(V6),1.680100e+0,e / N2,"E + N2 -> E + N2( VIB 6V1), Excitation","E = 1.6801 eV, complete set",,2012-10-11 09:34:55,N2,N2(V6)
7,8,N2,8,EXCITATION,N2 <-> N2(V7),1.947500e+0,e / N2,"E + N2 -> E + N2( VIB 7V1), Excitation","E = 1.9475 eV, complete set",,2012-10-11 09:34:55,N2,N2(V7)
8,9,N2,9,EXCITATION,N2 <-> N2(V8),2.211500e+0,e / N2,"E + N2 -> E + N2( VIB 8V1), Excitation","E = 2.2115 eV, complete set",,2012-10-11 09:34:55,N2,N2(V8)
9,10,N2,10,EXCITATION,N2 <-> N2(V9),2.471800e+0,e / N2,"E + N2 -> E + N2( VIB 9V1), Excitation","E = 2.4718 eV, complete set",,2012-10-11 09:34:55,N2,N2(V9)


#Debug

In [ ]:
filePath = '/content/drive/MyDrive/KAUST/bolsig/output(N2O2).txt'

In [ ]:
columns = ['no', 'state', 'criteria', 'value']

In [ ]:
f = open(filePath, 'r')
data = {c : list() for c in columns}
_line = '--------------------------------'
_line2 = '------------------------------------------------------------'

lines = f.readlines()
lineIdx = 0

while lineIdx < len(lines):
  line = lines[lineIdx]
  line = line.strip()

  if line == _line2: break

  if 'Input cross section' in line:
    data['no'].append(int(line.split(' ')[0][1:]))

    for i in range(1,4):
      lineIdx += 1
      line = lines[lineIdx]
      line = line.strip()

      if line == _line:
        data[columns[3]].append('')
        break
      data[columns[i]].append(line)

  lineIdx += 1
f.close()

df = pd.DataFrame(data)
df.to_csv(filePath.split('.')[0] + '(output).csv', index = False)

In [ ]:
df

,no,state,criteria,value
0,1,ELASTIC,N2,0.1950E-04
1,2,EXCITATION,N2,0.2889 1.000
2,3,EXCITATION,N2,0.5742 1.000
3,4,EXCITATION,N2,0.8559 1.000
4,5,EXCITATION,N2,1.134 1.000
5,6,EXCITATION,N2,1.409 1.000
6,7,EXCITATION,N2,1.680 1.000
7,8,EXCITATION,N2,1.948 1.000
8,9,EXCITATION,N2,2.212 1.000
9,10,EXCITATION,N2,2.472 1.000


In [ ]:
length = len(df)
data = {'output' : list(), 'output.' : list()}

for idx in range(length):
  row1 = df.loc[idx]
  row2 = choosedDf.loc[idx]

  for col in columns[1:]:
    if col == 'value':
      data1 = row1[col].split(' ')[0]
      data2 = row2[col].split(' ')[0]

      if data1 != '' and data2 != '':
        data['output'].append(float(data1))
        data['output.'].append(float(data2))
      else:
        print(data1, data2)

    elif row1[col] != row2[col]: print('error')

compareDf = pd.DataFrame(data)

In [ ]:
compareDf

,output,output.
0,0.000019,0.000019
1,0.288900,0.288900
2,0.574200,0.574200
3,0.855900,0.855900
4,1.134000,1.134200
5,1.409000,1.408800
6,1.680000,1.680100
7,1.948000,1.947500
8,2.212000,2.211500
9,2.472000,2.471800
